In [30]:
# generate query from user message
import sqlite3
from rasa_nlu.converters import load_data
from rasa_nlu.model import Trainer
from rasa_nlu.config import RasaNLUConfig

pipeline = [
    "nlp_spacy",
    "tokenizer_spacy",
    "ner_crf"
]

# Create a config that uses this pipeline
config = RasaNLUConfig(cmdline_args={"pipeline":pipeline})

# Create a trainer that uses this config
trainer = Trainer(config)

training_data = load_data("./training_data1.json")

# Create an interpreter by training the model
interpreter = trainer.train(training_data)


In [43]:
# Define find_hotels()
def find_hotels(params):
    # Create the base query
    query = 'SELECT * FROM hotels'
    # Add filter clauses for each of the parameters
    if len(params) > 0:
        filters = ["{}=?".format(k) for k in params]
        #print(filters)
        query += " WHERE " + " and ".join(filters)
    #    print(query)
    # Create the tuple of values
    t = tuple(params.values())
#    print(t,'Hello Bharat kumar')
    
    # Open connection to DB
    conn = sqlite3.connect("hotels.db")
    # Create a cursor
    c = conn.cursor()
    # Execute the query
    c.execute(query,t)
    # Return the results
    return c.fetchall()


In [44]:
# Create the dictionary of column names and values
params = {"area":'south',"price":'lo'}

# Find the hotels that match the parameters
print(find_hotels(params))


OperationalError: no such table: hotels

In [46]:
# Define respond()
def respond(message):
    # Extract the entities
    #print(message)
    #entities1=interpreter.parse(message)
    entities = interpreter.parse(message)["entities"]
    #print(entities1)
    #print()
    #print(type(entities))
    print(entities)
    
    
    # Initialize an empty params dictionary
    params = {}
    # Fill the dictionary with entities
    for ent in entities:
        params[ent["entity"]] = str(ent["value"])

    #print(params)     
    # Find hotels that match the dictionary
    results = find_hotels(params)
    # Get the names of the hotels and index of the response
    names = [r[0] for r in results]
    n = min(len(results),3)
    # Select the nth element of the responses array
    return responses[n].format(*names)
    
print(respond("I want an expensive hotel in the south of town"))

[{'start': 10, 'end': 19, 'value': 'expensive', 'entity': 'cuisine', 'extractor': 'ner_crf'}, {'start': 33, 'end': 38, 'value': 'south', 'entity': 'location', 'extractor': 'ner_crf'}]


OperationalError: no such table: hotels

In [47]:
# Define a respond function, taking the message and existing params as input
def respond(message,params):
    # Extract the entities
    entities = interpreter.parse(message)['entities']
    # Fill the dictionary with entities
    for ent in entities:
        params[ent["entity"]] = str(ent["value"])

    # Find the hotels
    results = find_hotels(params)
    names = [r[0] for r in results]
    n = min(len(results), 3)
    # Return the appropriate response
    return responses[n].format(*names), params

# Initialize params dictionary
params = {}

# Pass the messages to the bot
for message in ["I want an expensive hotel", "in the north of town"]:
    print("USER: {}".format(message))
    response, params = respond(message, params)
    print("BOT: {}".format(response))

USER: I want an expensive hotel


OperationalError: no such table: hotels

In [48]:
# Define negated_ents()
def negated_ents(phrase):
    # Extract the entities using keyword matching
    ents = [e for e in ["south", "north"] if e in phrase]
    # Find the index of the final character of each entity
    ends = sorted([phrase.index(e) + len(e) for e in ents])
    # Initialise a list to store sentence chunks
    chunks = []
    # Take slices of the sentence up to and including each entitiy
    start = 0
    for end in ends:
        chunks.append(phrase[start:end])
        start = end
    result = {}
    # Iterate over the chunks and look for entities
    for chunk in chunks:
        for ent in ents:
            if ent in chunk:
                # If the entity is preceeded by a negation, give it the key False
                if "not" in chunk or "n't" in chunk:
                    result[ent] =False
                else:
                    result[ent] = True
    return result  

# Check that the entities are correctly assigned as True or False
for test in tests:
    #print(test[0],test[1])
    print(negated_ents(test[0]) == test[1])


NameError: name 'tests' is not defined

In [51]:
import spacy
nlp=spacy.load('en')

In [52]:
doc = nlp('not sushi, maybe pizza?')

In [72]:
indices=[1,4]


In [2]:
'''
start=0
for i in  indices:
    print(doc[start:i],doc[i])
    start=i

'''


'\nstart=0\nfor i in  indices:\n    print(doc[start:i],doc[i])\n    start=i\n\n'

In [84]:
ents, negated_ents = [], []
start=0
for i in indices:
    phrase = "{}".format(doc[start:i])
#    print(phrase)
    print(doc[start:i])
    if "not" in phrase or "n't" in phrase:
              negated_ents.append(doc[i])
#              print(doc[i])
    else:
             ents.append(doc[i])
    start = i

print(ents,negated_ents)    

not
sushi, maybe
[pizza] [sushi]


In [86]:
tests=[("no I don't want to be in the south", {'south': False}), ('no it should be in the south', {'south': True}), ('no in the south not the north', {'south': True, 'north': False}), ('not north', {'north': False})]

In [87]:
# Define negated_ents()
def negated_ents(phrase):
    # Extract the entities using keyword matching
    ents = [e for e in ["south", "north"] if e in phrase]
    print(ents)
    
    # Find the index of the final character of each entity
    ends = sorted([phrase.index(e) + len(e) for e in ents])
    print(ends)
    
    # Initialise a list to store sentence chunks
    chunks = []
    
    
    # Take slices of the sentence up to and including each entitiy
    start = 0
    for end in ends:
        chunks.append(phrase[start:end])
        start = end
    result = {}
    # Iterate over the chunks and look for entities
    for chunk in chunks:
        for ent in ents:
            if ent in chunk:
                # If the entity is preceeded by a negation, give it the key False
                if "not" in chunk or "n't" in chunk:
                    result[ent] =False
                else:
                    result[ent] = True
    return result  

# Check that the entities are correctly assigned as True or False
for test in tests:
    print(test[0],test[1])
    print(negated_ents(test[0]) == test[1])


no I don't want to be in the south {'south': False}
['south']
[34]
True
no it should be in the south {'south': True}
['south']
[28]
True
no in the south not the north {'south': True, 'north': False}
['south', 'north']
[15, 29]
True
not north {'north': False}
['north']
[9]
True


In [88]:
    # Define the respond function
    def respond(message,params,neg_params):
        # Extract the entities
        entities = interpreter.parse(message)["entities"]
        ent_vals = [e["value"] for e in entities]
        # Look for negated entities
        negated = negated_ents(message,ent_vals)
        for ent in entities:
            if ent["value"] in negated and negated[ent["value"]]:
                neg_params[ent["entity"]] = str(ent["value"])
            else:
                params[ent["entity"]] = str(ent["value"])
        # Find the hotels
        results = find_hotels(params,neg_params)
        names = [r[0] for r in results]
        n = min(len(results),3)
        # Return the correct response
        return responses[n].format(*names), params, neg_params
    
    # Initialize params and neg_params
    params = {}
    neg_params = {}
    
    # Pass the messages to the bot
    for message in ["I want a cheap hotel", "but not in the north of town"]:
        print("USER: {}".format(message))
        response, params, neg_params = respond(message, params, neg_params)
        print("BOT: {}".format(response))


USER: I want a cheap hotel


TypeError: negated_ents() takes 1 positional argument but 2 were given